In [13]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone
import ipywidgets as widgets
from IPython.display import display, clear_output

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from dotenv import load_dotenv
import os
import numpy as np

load_dotenv()
API_URL = os.getenv("API_URL")
# %run "./news_source/news.ipynb" import News




In [37]:
class GoogleNews:

    def __init__(self, sectionId):
        self.sectionId = sectionId
        self.newsSource, self.sectionName, self.sectionArr, self.tags = self.get_news()
        
        # Dados da página inicial do Google News 
        self.html_soup = self.get_html(self.newsSource)
        self.main_news = self.get_main_news_list()
        
        # URLs da página secundária (selecionada pelo usuário), do Google News
        self.selected_main_news = ''
        self.output_area = widgets.Output()
        self.create_checkboxes()
        self.all_news_data


    def get_news(self):
        url = f'{API_URL}/api/newssource'
        newssource = np.array(requests.get(url).json())
        result = [item for item in newssource if item['__t'] == 'google news - html' and item['sectionId'] == '66ad2aa7b591d67921fadc4b'][0]
        return result['newsSource'], result['sectionName'], result['sectionArr'], result['tags']
    
    
    def get_html(self, url):
        browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        html_content = requests.get(url, headers = browsers)
        soup = BeautifulSoup(html_content.text, 'html')
        return soup
    
    
    def get_main_news_list(self):
        news_blocks = self.html_soup.find_all (class_='PO9Zff Ccj79 kUVvS')[0]['url']
        result = []
        for nb in news_blocks:
            try:
                title = nb.find('a', class_='gPFEn').get_text()
                all_news = nb.find('div', class_='Ylktk').find('a').get('href')
                result.append({'title': title, 'url': all_news})
            except:
                pass
        return result


    def create_checkboxes(self):
        options = [news['title'] for news in self.main_news]
        self.news_checkboxes = widgets.Dropdown(
            options = options,
            description=f'{self.sectionName}:',
            disabled=False,
            layout=widgets.Layout(width='70%')
        )
        
        self.confirm_button = widgets.Button(
            description='Gerar notícias',
            button_style='success',
            tooltip='Clique para confirmar as notícias selecionadas',
            icon='check',
            layout=widgets.Layout(width='20%')
        )
        self.confirm_button.on_click(self.on_confirm_button_click)
        display(self.news_checkboxes, self.confirm_button, self.output_area)
    
    
    def on_confirm_button_click(self, b):
        with self.output_area:
            clear_output()
            selected_title = self.news_checkboxes.value
            self.selected_main_news = self.main_news[[news['title'] for news in self.main_news].index(selected_title)]
            print(f"Notícia selecionada: {self.selected_main_news}")
    
    
    # Buscar dados da página secundária do Google News
    def get_all_news_data(self):
        soup = self.get_html(self.selected_main_news['url'])
        
        initialUrls = []
        news_all = self.html_soup.find_all('entry')
        for news in news_all:
            initialUrl = news.find('link').get('href')
            initialUrls += [initialUrl]
        
        if not self.initial_url or not self.initial_url.startswith(('http://', 'https://')):
            raise ValueError(f"URL inválida: {self.initial_url}")
        
        finalUrls=[]
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        try:
            for url in initialUrls:
                driver.get(url)
                WebDriverWait(driver, 15).until(EC.url_changes(url))
                finalUrl = driver.current_url
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
        except:
            print("O redirecionamento não ocorreu dentro do tempo esperado.")
        driver.quit()
            
            
            
            
        news_data = News(url)
        return result
    
    





In [40]:
# gn = GoogleNews('https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNREUxWm5JU0JYQjBMVUpTS0FBUAE?hl=pt-BR&gl=BR&ceid=BR%3Apt-419', 'g1', 'Brasil', ['brasil', 'mundo', 'tecnologia', 'ciencia'], ['brasil', 'mundo', 'tecnologia', 'ciencia'])

